# UV-Visible Spectrophotometry — Interactive Simulation


In [ ]:
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
import ipywidgets as W
from IPython.display import display, clear_output

MOLECULES = {
    'DNA 260 nm': (260, 15000, 20),
    'Protein 280 nm': (280, 5500, 18),
    'NADH 340 nm': (340, 6220, 12),
    'Heme 414 nm (Soret)': (414, 100000, 8),
    'GFP 488 nm': (488, 55000, 25),
}

wl = np.linspace(220, 700, 1000)
def gaussian(x, mu, sigma, amp): return amp*np.exp(-(x-mu)**2/(2*sigma**2))

def simulate(molecule, conc_uM, path_cm, baseline, noise):
    lam, eps, width = MOLECULES[molecule]
    sigma = width/2.355
    absorb = gaussian(wl, lam, sigma, 1.0)
    A = absorb * eps * (conc_uM*1e-6) * path_cm
    A += baseline
    if noise>0: A += np.random.normal(0, noise, size=wl.shape)
    return A

mol = W.Dropdown(options=list(MOLECULES), value='DNA 260 nm', description='Molecule')
conc = W.FloatLogSlider(value=10, base=10, min=-1, max=3, step=0.1, description='Conc (µM)')
path = W.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Path (cm)')
baseline = W.FloatSlider(value=0.0, min=-0.1, max=0.2, step=0.005, description='Baseline')
noise = W.FloatSlider(value=0.005, min=0, max=0.05, step=0.001, description='Noise')
out = W.Output()

def update(*_):
    with out:
        clear_output(wait=True)
        A = simulate(mol.value, conc.value, path.value, baseline.value, noise.value)
        plt.figure(figsize=(7,4))
        plt.plot(wl, A)
        plt.xlabel('Wavelength (nm)'); plt.ylabel('Absorbance (A)')
        plt.title(f'{mol.value} — conc={conc.value:.2g} µM, path={path.value:.1f} cm')
        plt.ylim(-0.1, max(1.2*np.max(A), 0.6)); plt.grid(True); plt.show()

for w in [mol, conc, path, baseline, noise]: w.observe(update, 'value')
display(W.VBox([W.HBox([mol, conc]), W.HBox([path, baseline, noise]), out])); update()
